# String Matching with genetic algorithm

Manuel Arias - Juan Camilo Sarimento

In [13]:
import numpy as np
import string

In [14]:
def init_populations(chrom_length,popu_length,alphabet):
    individuals = []
    for i in range(popu_length):
        chromosome = []
        for j in range(chrom_length):
            p = np.random.randint(0,len(alphabet))
            chromosome.append(alphabet[p])
        individuals.append(chromosome)
    return individuals

In [15]:
alphabet=string.ascii_letters+" ,."
ch1=init_populations(32,10,alphabet)
ch1[0]

['I',
 'W',
 ' ',
 'E',
 'e',
 'W',
 'D',
 'O',
 ',',
 'E',
 'V',
 'y',
 'P',
 'j',
 'n',
 'T',
 '.',
 'H',
 'L',
 'y',
 'H',
 'n',
 'S',
 'C',
 'R',
 'A',
 'r',
 'q',
 't',
 'm',
 'R',
 'P']

In [16]:
# def decode(chromosome):
#     decoded=[]
#     for letter in chromosome:
#         binary_array = letter.to_bytes(1, "big")
#         ascii_text = binary_array.decode()
#         decoded.append(ascii_text)
#     decoded="".join(decoded)
#     return decoded

In [17]:
#decode(ch1[0])

In [42]:
def fitness(chromosome,target):
    # decoded_chromosome=decode(chromosome)
    count=0
    for i in range(len(target)):
        # if decoded_chromosome[i]==target[i]:
        if chromosome[i]==target[i]:
            count+=1
            
    return count

In [19]:
target="Me llamo Juan y mi compa Manuel."
fitness(ch1[0],target)

1

In [20]:
def prob_fitness(pop,target):
    probs = []
    cumulative_prob = []
    fitnesses = []
    acc = 0

    for i in range(len(pop)):
        fitnesses.append( fitness(pop[i],target) )
        acc += fitnesses[i]
        cumulative_prob.append(acc)

    for j in range(len(pop)):
        if acc!=0:
            probs.append(fitnesses[j]/acc)
        else:
            probs.append(0)

    return probs, cumulative_prob,fitnesses

In [21]:
target="hola"
prob_fitness(["pola","foca",target,target,target],target)

([0.17647058823529413,
  0.11764705882352941,
  0.23529411764705882,
  0.23529411764705882,
  0.23529411764705882],
 [3, 5, 9, 13, 17],
 [3, 2, 4, 4, 4])

In [22]:
def roullete(cumulative_fit,pop):
    selected_ind = []

    for i in range(2):
        ran = np.random.rand() * cumulative_fit[-1]

        s = 0

        while ran > cumulative_fit[s]:
            s += 1

        selected_ind.append(pop[s])
    return selected_ind

In [23]:
def select_by_fitness(fitness,pop,parent_size):
    sorted_ind=sorted(range(len(fitness)), key=fitness.__getitem__,reverse=True)    #O(n log n)
    # sorted_ind=np.argsort(fitness)
    sorted_pop=[]
    for i in range(parent_size):
        sorted_pop.append(pop[sorted_ind[i]])
    # sorted_pop=pop[sorted_ind]
    # return sorted_pop[:parent_size]
    return sorted_pop

In [24]:
def crossover(p_cross, cumulative_fit, popu_length,pop,chrom_length,fit,parent_size):
    
    new_gen = []
    parents=select_by_fitness(fit,pop,parent_size)
    diff_parents=False

    while len(new_gen)<popu_length:
        
        # selected_ind = roullete(cumulative_fit,pop)
        # ind1 = selected_ind[0]
        # ind2 = selected_ind[1]
        while not diff_parents:
            x1=np.random.randint(0,len(parents))
            x2=np.random.randint(0,len(parents))
            if x1!=x2:
                diff_parents=True
        ind1=parents[x1]
        ind2=parents[x2]
            

        p = np.random.rand()

        if p < p_cross:

            cut = np.random.randint(1,chrom_length-1)
            # print(cut)
            new_ind1 = ind1[:cut] + ind2[cut:]
            new_ind2 = ind2[:cut] + ind1[cut:]
            # print("new_ind1",decode(new_ind1))
            # print("new_ind2",decode(new_ind2))
            new_gen.append(new_ind1)
            new_gen.append(new_ind2)
            # print("\n")
    return new_gen

In [26]:
probs, cumulative_prob,fitnesses=prob_fitness(ch1,target)
p_cross , popu_length , chrom_length,parent_size = 0.9 , 10 , 32 , 4
new_gen=crossover(p_cross, cumulative_prob, popu_length,ch1,chrom_length,fitnesses,parent_size)

In [27]:
def mutation(new_gen,p_mutation,chrom_length,alphabet):
    for i in range(len(new_gen)):
        # print("before",new_gen[i])
        for j in range(chrom_length):
            p= np.random.rand()
            if p<p_mutation:
                letter=np.random.randint(0,len(alphabet))
                new_gen[i][j]=alphabet[letter]
                # letter=np.random.randint(32,126)
                # if new_gen[i][j]!=letter:
                #     new_gen[i][j]=letter
                # else:
                    # # while new_gen[i][j]!=letter:
                    # while new_gen[i][j]!=alphabet[letter]:
                    #     letter=np.random.randint(0,len(alphabet))
                    #     new_gen[i][j]=alphabet[letter]
                    #     # letter=np.random.randint(32,126)
                    #     # new_gen[i][j]=letter
        # print("after",new_gen[i])
    return new_gen

In [28]:
new_mutated=mutation(new_gen,0.04,chrom_length,alphabet)

In [58]:
def genetic_algo(target,alphabet,p_cross,p_mutation,popu_length,chrom_length,number_gen,parent_size):#,random_state):
    # np.random.seed(random_state)
    
    min, max, mean, best_chrom = [], [],[], []

    pop = init_populations(chrom_length,popu_length,alphabet)
    probs, cumulative_fit, fitnesses = prob_fitness(pop,target)
    print("fitnesses gen 0:",fitnesses)
    best_chrom.append(pop[np.argmax(fitnesses)])
    
    min.append(np.min(fitnesses))
    max.append(np.max(fitnesses))
    mean.append(np.mean(fitnesses))

    new_gen = crossover(p_cross, cumulative_fit, popu_length,pop,chrom_length,fitnesses,parent_size)
    new_gen = mutation(new_gen,p_mutation,chrom_length,alphabet)
    best_chrom.append(new_gen[np.argmax(fitnesses)])
    min.append(np.min(fitnesses))
    max.append(np.max(fitnesses))
    mean.append(np.mean(fitnesses))

    for i in range(number_gen-1):
        new_gen = crossover(p_cross, cumulative_fit, popu_length,new_gen,chrom_length,fitnesses,parent_size)
        new_gen = mutation(new_gen,p_mutation,chrom_length,alphabet)
        probs, cumulative_fit, fitnesses = prob_fitness(new_gen,target)

        best_chrom.append(new_gen[np.argmax(fitnesses)])
        # if (i+1)%50==0:
        #     print(f"Generation {i+1}: {best_chrom[-1]}")
        print(f"Generation {i+1}: {new_gen[np.argmax(fitnesses)]}, fitness:{fitnesses[np.argmax(fitnesses)]}")
        min.append(np.min(fitnesses))
        max.append(np.max(fitnesses))
        mean.append(np.mean(fitnesses))
        if fitnesses[np.argmax(fitnesses)]==len(target):
            break

    return min, max, mean, best_chrom

##### Complexity of genetic algorithm:

init_populations: O(p*c) where p is popu_length and c is chrom_length. Each chromosome is created by character and then added to pop.

prob_fitness: O(p*c)  where p is popu_length and c is chrom_length. For each individual in pop, fitness is calculated by counting over each character in the target string

select_by_fitness: O(n) where n is parent size. (sorted buit in function used has complexity O(p*logp) with p, popu_length). There is a for loop over the lenth of the parent size.

crossover: O(p) where p is popu_length. There is a while loop that iterates over the population to create the new generation

mutation: O(p*c) where p is popu_length (new_gen length), and c is chrom_length

genetic_algo: 

init_populations + prob_fitness + crossover + mutation + O((g-1)*(prob_fitness + crossover + mutation))

=O(g\*p\*c), where g is number_gen, p is popu_lenth and c is chrom_length

In [59]:
p_cross=0.9
popu_length=100
p_mutation=0.01
number_gen=2000
rand_state=42
target="hola"
chrom_length=len(target)
# alphabet=string.ascii_letters+" ,."
alphabet=string.ascii_lowercase
parent_size=10

min, max, mean, best_chrom = genetic_algo(target,alphabet,p_cross,p_mutation,popu_length,chrom_length,number_gen,parent_size)#,rand_state)

fitnesses gen 0: [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Generation 1: ['u', 'r', 'l', 'i'], fitness:1
Generation 2: ['u', 'r', 'l', 'r'], fitness:1
Generation 3: ['u', 'r', 'l', 'i'], fitness:1
Generation 4: ['u', 'r', 'l', 'i'], fitness:1
Generation 5: ['u', 'r', 'l', 'i'], fitness:1
Generation 6: ['h', 'r', 'l', 'i'], fitness:2
Generation 7: ['u', 'r', 'l', 'i'], fitness:1
Generation 8: ['u', 'r', 'l', 'i'], fitness:1
Generation 9: ['u', 'r', 'l', 'i'], fitness:1
Generation 10: ['u', 'r', 'l', 'i'], fitness:1
Generation 11: ['u', 'r', 'l', 'i'], fitness:1
Generation 12: ['u', 'r', 'l', 'i'], fitness:1
Generation 13: ['u', 'r', 'l', 'i'], fitness:1
Generation 14: ['u', 'r', 'l', 'i'], fitness:1
Generation 15: ['u', 'r', 'l', 'i